# Introduction
Train a bert classifier using tensorflow hub. Adapted from:

[1]TensorFlow code and pre-trained models for BERT. Contribute to google-research/bert development by creating an account on GitHub. Google AI Research, 2019.


In [1]:
from platform import python_version
print("VERSION: ", python_version()) # expect 3.7.0

VERSION:  3.7.0


In [2]:
import json
from pprint import pprint
import os
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import importlib

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from imblearn.over_sampling import SMOTE

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# custom data loading functions
import load_data
import clean_data
import jupyter_bert

W0402 13:29:45.530753 140736101036928 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
# dirty spaCy error workaround:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [4]:
# test = '../data/data_turk/dummy_data.json'
annotations = '../data/data_turk/Annotations04-02-19.json'

### Load Data

In [5]:
importlib.reload(load_data)
df = load_data.getJSONData(annotations)
df.head()

,annotation,fileID,text
0,permission_statement,370,"""yes, i consent parent/legal guardian signature:"""
1,NON_permission_statement.,370,/ my child has already had dtpa vaccination i ...
2,NON_permission_statement.,370,"""prior to administering the vaccine(s), the nu..."
3,NON_permission_statement.,370,""". - within 2 business days of immunisation, i..."
4,NON_permission_statement.,370,mobile email pre vaccination checklist (please...


### Reformat Data for Bert Model

In [6]:
to  = 'label'
field = 'annotation'

df[to] = df.apply(lambda row:clean_data.convertAnnotationtoBinary(row,field), axis =1) 
df.head()

,annotation,fileID,text,label
0,permission_statement,370,"""yes, i consent parent/legal guardian signature:""",1
1,NON_permission_statement.,370,/ my child has already had dtpa vaccination i ...,0
2,NON_permission_statement.,370,"""prior to administering the vaccine(s), the nu...",0
3,NON_permission_statement.,370,""". - within 2 business days of immunisation, i...",0
4,NON_permission_statement.,370,mobile email pre vaccination checklist (please...,0


In [7]:
to  = 'text_a'
field = 'text'
    
df[to] = df.apply(lambda row:clean_data.cleanSents(row, field), axis=1)
df.head()

,annotation,fileID,text,label,text_a
0,permission_statement,370,"""yes, i consent parent/legal guardian signature:""",1,yes i consent parent legal guardian signature
1,NON_permission_statement.,370,/ my child has already had dtpa vaccination i ...,0,my child has already had dtpa vaccination i do...
2,NON_permission_statement.,370,"""prior to administering the vaccine(s), the nu...",0,prior to administering the vaccine s the nurse...
3,NON_permission_statement.,370,""". - within 2 business days of immunisation, i...",0,within 2 business days of immunisation i under...
4,NON_permission_statement.,370,mobile email pre vaccination checklist (please...,0,mobile email pre vaccination checklist please ...


In [8]:
df['text_b'] = 'a'
df.head()

,annotation,fileID,text,label,text_a,text_b
0,permission_statement,370,"""yes, i consent parent/legal guardian signature:""",1,yes i consent parent legal guardian signature,a
1,NON_permission_statement.,370,/ my child has already had dtpa vaccination i ...,0,my child has already had dtpa vaccination i do...,a
2,NON_permission_statement.,370,"""prior to administering the vaccine(s), the nu...",0,prior to administering the vaccine s the nurse...,a
3,NON_permission_statement.,370,""". - within 2 business days of immunisation, i...",0,within 2 business days of immunisation i under...,a
4,NON_permission_statement.,370,mobile email pre vaccination checklist (please...,0,mobile email pre vaccination checklist please ...,a


In [9]:
df = df.rename(index=str, columns={"fileID": "guid"})

### Explore Some Positive Cases

In [ ]:
positives = df.loc[df['label'] == 1]
positives["guid"] = pd.to_numeric(positives["guid"])
positives = positives.rename(index=str, columns={"guid": "fileID"})
positives.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,annotation,fileID,text,label,text_a,text_b
0,permission_statement,370,"""yes, i consent parent/legal guardian signature:""",1,yes i consent parent legal guardian signature,a
7,permission_statement,490,"""if you sign your name below, it means that yo...",1,if you sign your name below it means that you ...,a
15,permission_statement,565,i also understand that i should not consume al...,1,i also understand that i should not consume al...,a
19,permission_statement,387,"""if you agree to being audiotaped but feel unc...",1,if you agree to being audiotaped but feel unco...,a
36,permission_statement,515,we may need to re- contact you if we are study...,1,we may need to re contact you if we are studyi...,a


In [ ]:
filenames = pd.read_csv('../data/outputs/file_ids-2019-03-26.csv')
filenames = filenames.drop(['path'], axis=1)
filenames.head()

,fileID,name
0,1,TAMU - HRPP Informed consent.txt
1,2,Potomac Primary Care_flu-consent-form.txt
2,3,OSU_Scheduled_Delivery_Consent.txt
3,4,consent_biorepository_12-19-14.txt
4,5,Cambridge_Consent_endodontics2.txt


In [ ]:
joined = pd.merge(positives, filenames, on='fileID', how='outer')
joined.rename(index=str, columns={"name": "file_name"})
joined = joined.drop(['text', 'label', 'text_b', 'fileID'], axis=1)
joined.head(10)

,annotation,text_a,name
0,permission_statement,yes i consent parent legal guardian signature,Queensland_AUSTRALIA_ Immunization Consent For...
1,permission_statement,if you sign your name below it means that you ...,MinorAssentForm030708 (1).txt
2,permission_statement,i also understand that i should not consume al...,Cambridge_Consent_endodontics11.txt
3,permission_statement,if you agree to being audiotaped but feel unco...,CF-Sample_Interview_Audiotape.txt
4,permission_statement,we may need to re contact you if we are studyi...,Marshfield-Consent-Form-10.28.13.txt
5,permission_statement,if you become ill or injured from this study m...,Marshfield-Consent-Form-10.28.13.txt
6,permission_statement,in order to minimize the risk of unintended re...,Marshfield-Consent-Form-10.28.13.txt
7,permission_statement,while we work with other researchers that are ...,Marshfield-Consent-Form-10.28.13.txt
8,permission_statement,your relatives will not be contacted because o...,Marshfield-Consent-Form-10.28.13.txt
9,permission_statement,we will code all information entered into the ...,Marshfield-Consent-Form-10.28.13.txt


In [ ]:
# joined.to_csv('../data/outputs/positive_class_examples.csv', index=False)

### Sample to Speed it up.

In [ ]:
# n_samples = 500
# df = df.sample(n=n_samples, random_state=1729)

In [ ]:
cols = ['guid', 'text_a', 'text_b', 'label', ] 
df = df[cols]
df.head()

,guid,text_a,text_b,label
0,370,yes i consent parent legal guardian signature,a,1
1,370,my child has already had dtpa vaccination i do...,a,0
2,370,prior to administering the vaccine s the nurse...,a,0
3,370,within 2 business days of immunisation i under...,a,0
4,370,mobile email pre vaccination checklist please ...,a,0


### Train - Test split

In [ ]:
train, test = train_test_split(df,stratify=df['label'],test_size=0.3)

print(type(train), type(test))
print(train.dtypes)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
guid      object
text_a    object
text_b    object
label      int64
dtype: object


### SMOTE IT
Unfortunatelty SMOTE does not work for this kind of data as implemented. Need to kleep thinking.

In [ ]:
# # very annoyingly, SMOTE implementation does not support passing text. 
# # here's a dumb workaround.

# sm = SMOTE(random_state=12, ratio = 1.0)

# train['index1'] = train.index
# train_prep = train[['guid', 'index1']]

# sm_train_idx, sm_train_label = sm.fit_sample(train_prep,train['label'])

# print(len(train_prep), len(train['label']))
# print(len(sm_train_idx), len(sm_train_label))

In [ ]:
# # combine the smote results with the text data

# smote_results = pd.DataFrame({'guid':sm_train_idx[:,0],
#                                   'index1':sm_train_idx[:,1]})

# smote_results['label'] = sm_train_label
# smote_results['guid'] = smote_results['guid'].astype(object)

# rejoin_from_train_df = train[['text_a', 'text_b', 'index1']]
# rejoin_from_train_df['index1'] = rejoin_from_train_df['index1'].astype(float)

# rejoin = pd.merge(smote_results, rejoin_from_train_df, on='index1', how='outer')
# rejoin = rejoin.set_index('index1')
# rejoin['text_b'] = 'a'
# cols = ['guid', 'text_a', 'text_b', 'label'] 
# rejoin = rejoin[cols]
# train = rejoin

In [ ]:
# for idx, row in train.iterrows():
#     print(row, '\n')

In [ ]:
DATA_COLUMN = 'text_a'
LABEL_COLUMN = 'label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [ ]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

### BERT config

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
OUTPUT_DIR = '../bert_output' + today
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: ../bert_output2019-04-02 *****


In [ ]:
tokenizer = jupyter_bert.create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0402 13:29:56.319947 140736101036928 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0402 13:29:58.477382 140736101036928 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [ ]:
# # test
# tokenizer.tokenize("This here's an example of using the BERT tokenizer")

In [ ]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 1942


I0402 13:29:58.950724 140736101036928 run_classifier.py:774] Writing example 0 of 1942


INFO:tensorflow:*** Example ***


I0402 13:29:58.953509 140736101036928 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0402 13:30:00.063749 140736101036928 run_classifier.py:466] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0402 13:30:00.064900 140736101036928 run_classifier.py:467] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0402 13:30:00.066213 140736101036928 run_classifier.py:468] label: 0 (id = 0)


In [ ]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = jupyter_bert.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '../bert_output2019-04-02', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x157865080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0402 13:30:00.546468 140736101036928 estimator.py:201] Using config: {'_model_dir': '../bert_output2019-04-02', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x157865080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

INFO:tensorflow:Calling model_fn.


I0402 13:30:02.026859 140736101036928 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0402 13:30:05.690821 140736101036928 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

I0402 13:30:20.509099 140736101036928 session_manager.py:493] Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 32 into ../bert_output2019-04-02/model.ckpt.


I0402 13:30:28.492151 140736101036928 basic_session_run_hooks.py:594] Saving checkpoints for 32 into ../bert_output2019-04-02/model.ckpt.


INFO:tensorflow:loss = 0.22138637, step = 33


I0402 13:31:07.095727 140736101036928 basic_session_run_hooks.py:249] loss = 0.22138637, step = 33


### Bert eval

In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

## (OBSOLETE) 
This section contains code to output annotation data into files for local invocation of BERT.

In [ ]:
# # remove old files
# os.remove('../data/bert_training_inputs/train.tsv')
# os.remove('../data/bert_training_inputs/dev.tsv')
# os.remove('../data/bert_training_inputs/test.tsv')

In [ ]:
# # split into train, test, and dev 
# train, dummy = train_test_split(df, test_size=0.2)
# dev, test = train_test_split(dummy, test_size=0.5)

# print('original: ', len(df))
# print('train: ', len(train))
# print('dev: ', len(dev))
# print('test: ', len(test))

In [ ]:
# # Train to CSV
# train.insert(0, 'New_ID', range(0, 0 + len(train)))
# train.to_csv('../data/bert_training_inputs/train.tsv', sep='\t', index=False, header=False)
# train.head()

In [ ]:
# # Dev to CSV
# dev.insert(0, 'New_ID', range(0, 0 + len(dev)))
# dev.to_csv('../data/bert_training_inputs/dev.tsv', sep='\t', index=False, header=False)
# dev.head()

In [ ]:
# # Test to csv - requires a little more work 
# test.insert(0, 'New_ID', range(0, 0 + len(test)))
# cols = ['New_ID', 'clean_text']
# test = test[cols]
# test.columns = ['id', 'sentence']
# test.to_csv('../data/bert_training_inputs/test.tsv', sep='\t', index=False, header=True)
# test.head()